# Bootstrap, L=10

In [2]:
import sys
sys.path.insert(0, '../../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap, isomery_to_kraus
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic, channel_fidelity
from quantum_tools import apply_unitary, measurement, resample, state_fidelity
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix, IdealSPAM
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, complex_spacing_ratio, Spectrum, coat_spectrum, spectrum_to_angular, spectrum_to_radial
from quantum_circuits import integrable_circuit


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
def load_data(filename, n, seed, L):
    with open(filename, 'rb') as f:
        data = pkl.load(f)


    data = marginalize_counts(data, 0)

    targets = counts_to_probs(data)
    targets_spam = targets[:6**n]
    targets_map = targets[6**n:]

    np.random.seed(seed)
    
    circuit_target = integrable_circuit(n+1, L)
    unitary = Operator(circuit_target).data

    inputs_spam, _ = generate_pauliInput_circuits(n)
        
    inputs_map, circuit_list_map = (
                generate_pauli_circuits(n, None, N=5000-6**n)
        )
    
    return inputs_spam, targets_spam, inputs_map, targets_map, unitary

def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue()], sample_freq=100),
                )

    model.train(inputs = inputs,
                targets = targets,
                inputs_val = [inputs],
                targets_val = [targets],
                num_iter = num_iter,
                N = 500,
                verbose=verbose
                )
    
    return model

In [4]:
path = '../data/chaos_exp_reorder/5q/integrable_L=10/'
n = 4
d = 2**n
L = 10
bootstrap_samples = 10

for i in range(6, 10):

    spam_list = []
    model_list = []

    seed = 42 + i

    inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

    tf.random.set_seed(seed)
    for j in range(bootstrap_samples):
        targets_spam_bs = resample(targets_spam, 12000)
        targets_map_bs = resample(targets_map, 12000)

        spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
        spam_list.append(spam_model) 


        model = fit_model(inputs_map, 
                        targets_map_bs, 
                        spam_model,
                        num_iter = 3000, 
                        verbose=False)
        model_list.append(model)


    model_saver(spam_list, f'models/integrable_spam_L=10_bootstrap_seed{seed}.model')
    model_saver(model_list, f'models/integrable_model_L=10_bootstrap_seed{seed}.model')

[0.9933812206794932]
[0.9934183820817457]
[0.9933098537378943]
[0.9934218984208263]
[0.9933216474754694]
[0.9934136711223489]
[0.9933696661704385]
[0.9933937923334402]
[0.9933719862351176]
[0.993327971547825]
[0.9949603193279309]
[0.9949972522564394]
[0.9949943570265882]
[0.9949492154826456]
[0.994984517566755]
[0.995015492187395]
[0.9950002605828332]
[0.9950016624774118]
[0.9949378551705901]
[0.9949892769079224]
[0.9945783094061021]
[0.99457372776104]
[0.9945655604779116]
[0.9945446379995316]
[0.9945644308891617]
[0.9946010103057378]
[0.9946127489955656]
[0.9945596768982066]
[0.9945452813997872]
[0.9944928462274095]
[0.9944157969312809]
[0.9944591817755122]
[0.9944191465771901]
[0.9943468805328309]
[0.994473242192229]
[0.9943962610357492]
[0.9943984921967057]
[0.9943297783120203]
[0.994398414672741]
[0.9943654812321848]
